In [1]:
import numpy as np
from scipy.optimize import minimize
import math

In [2]:
def Gaussian(x1, x2, gamma):
    #Guassian kernel definition
    return math.exp((-1) * np.linalg.norm(x1 - x2, 2) / gamma)

def kernel_func(x1,x2,gamma):
        Kernel=np.zeros((x1.shape[0], x2.shape[0]))
        for i in range(x1.shape[0]):
            for j in range(x2.shape[0]):
                Kernel[i,j]=Gaussian(x1[i],x2[j],gamma)
        return Kernel

In [3]:
def main_function(alpha, Mat):
    
    return 0.5 * np.dot(alpha, np.dot(Mat, alpha)) - np.sum(alpha)

In [4]:
def jac(alpha, Mat):
    
    return np.dot(alpha.T, Mat) - np.ones(alpha.shape[0])

In [5]:
def kernel_svm(train_x, train_y, C,epoch, gamma):
    
    num_data=train_x.shape[0]
    
    # set parameters
    x0 = np.random.rand(num_data)
    bnds = [(0, C)] * num_data
    Mat = np.zeros((num_data, num_data))
    
    def Mat_func(train_x,train_y,gamma):
        for i in range(num_data):
            for j in range(num_data):
                Mat[i,j]=Gaussian(train_x[i],train_x[j],gamma)*train_y[i]*train_y[j]
        return Mat
    
    Mat=Mat_func(train_x,train_y,gamma)
    # optimize
    res = minimize(main_function, x0, args=(Mat,), method='L-BFGS-B', jac=jac, bounds=bnds)

    # w, b,alpha
    alpha=res.x
    
    Kernel=kernel_func(train_x,train_y,gamma)
    Kernel=Kernel*alpha*train_y
    
    w = np.sum([res.x[i] * train_y[i] * train_x[i,:] for i in range(num_data)], axis=0)
    b = np.mean(train_y - np.sum(Kernel, axis=0))

    return w, b,alpha

In [6]:
def kernel_test(x,y,train_x,train_y,alpha,b,gamma):
    
    error=0
    num_data=x.shape[0]
    Kernel=kernel_func(train_x,x,gamma)
    alpha = np.reshape(alpha, (alpha.shape[0], 1))
    train_y = np.reshape(train_y, (train_y.shape[0], 1))
    
    Kernel = Kernel*alpha * train_y 
    for i in range(num_data):
        tem=np.sign((np.sum(Kernel, axis=0) + b)[i])
        if tem!=y[i]:
            error+=1
    return error/num_data

In [7]:
train_data = []
with open('train.csv', 'r') as f:
    for term in f:
        train_data.append(term.strip().split(','))
        
test_data= []
with open('test.csv', 'r') as f:
    for term in f:
        test_data.append(term.strip().split(','))

In [8]:
train_data = np.array(train_data, dtype='float64')
test_data = np.array(test_data, dtype='float64')

In [9]:
train_x= train_data[:, :-1]
train_y = train_data[:, -1].astype(int)
# convert y label with -1,1
train_y[train_y == 0] = -1  

In [10]:
test_x= test_data[:, :-1]
test_y = test_data[:, -1].astype(int)
# convert y label with -1,1
test_y[test_y == 0] = -1 

In [11]:
gamma_set = [0.1, 0.5, 1, 5, 100]

In [13]:
#Q3.b
C_set =  [100.0/873, 500.0/873, 700.0/873]
for C in C_set:
    for g in gamma_set: 
        w, b, alpha = kernel_svm(train_x, train_y, epoch=100, C=C, gamma=g)
        print('C:', C, 'gamma:', g, 'weights:', w, 'bias:', b)
        # support vectors
        print('# support vectors:', np.sum(alpha != 0.0))
        # calculate training and test errors
        train_error = kernel_test(train_x, train_y,train_x, train_y, alpha, b, gamma=g)
        test_error = kernel_test(train_x, train_y,train_x, train_y, alpha, b, gamma=g)
        print('training error:',  train_error, 'test error', test_error)

C: 0.1145475372279496 gamma: 0.1 weights: [-204.6327895  -280.24352171   45.18617664   11.61509345] bias: -0.10779731423261878
# support vectors: 872
training error: 0.0 test error 0.0
C: 0.1145475372279496 gamma: 0.5 weights: [-204.5903865  -280.13537712   45.1437178    11.56933734] bias: -0.06482548359998565
# support vectors: 872
training error: 0.0 test error 0.0
C: 0.1145475372279496 gamma: 1 weights: [-155.65215833 -222.20654584   41.40953402    8.94456447] bias: -0.013411721580710415
# support vectors: 777
training error: 0.0 test error 0.0
C: 0.1145475372279496 gamma: 5 weights: [-47.8116733  -53.93136701   7.25562986  -1.42818775] bias: -0.8418853211033555
# support vectors: 323
training error: 0.06077981651376147 test error 0.06077981651376147
C: 0.1145475372279496 gamma: 100 weights: [-145.14649289 -137.22956314  -17.67891807  -11.75211185] bias: -0.4285875996787792
# support vectors: 731
training error: 0.11353211009174312 test error 0.11353211009174312
C: 0.572737686139748